In [75]:
#we need to use curl command to download the dataset from the roboflow and also specifying the unique id
!curl -L "your key" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

curl: (3) Host name 'your key' contains bad letter
Archive:  roboflow.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of roboflow.zip or
        roboflow.zip.zip, and cannot find roboflow.zip.ZIP, period.


In [68]:
#we will use a pretrained MobileNetV2 classifier for custom requirement
#The pretrained models are available in the library torchvision.models

from torchvision import models
from torchsummary import summary
import torch
from torch import nn
from torch.optim import Adam

In [41]:
custom_model = models.mobilenet_v2(pretrained=True)

In [42]:
print(custom_model)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

In [30]:
#since the above corrsponds to imagenet setup, we can see the last layer has 1000 classes
#And also we can either freeze the already trained model layers or unfreeze them
#This can be done by giving the command requires_grad = False, means layers are frozen
#for the above there is no training happens

for param in custom_model.parameters():
  param.requires_grad = False

#since it is a binary classification, the output will have two classes 

#if we have a GPU available, then we can use cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [52]:
num_ftrs = custom_model.classifier[1].in_features
print('number of input features to the last linear layer:', num_ftrs)

#now we need to update the last layer to output only two classes
custom_model.classifier[1] = nn.Linear(num_ftrs, 2)

number of input features to the last linear layer: 1280


In [55]:
print(custom_model)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

From the above definition we can see the output size is 2 corresponding to two categories.

In [61]:
#converting to cuda format
custom_model =  custom_model.to(device)

In [63]:
#setting up dataloader(multiple smaller batches). Since we do not process all the training
#samples at one go, we use dataloaders for processing multiple data chunks
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Resize

In [64]:
#define the path details from where the images should be retried
train_dataset = datasets.ImageFolder(root = '/content/train', transform = transforms.Compose([Resize((224,224)),ToTensor()]))
val_dataset = datasets.ImageFolder(root='/content/valid', transform=transforms.Compose([Resize((224,224)),ToTensor()]))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle=True)

In [69]:
#for loss function we can use the cross entropy function
loss_func = nn.CrossEntropyLoss()
opt = Adam(custom_model.parameters(), lr = 0.001)

In [73]:
epochs = 10

for e in range(0, epochs):
  custom_model.train()

  totalTrainLoss = 0
  totalValLoss = 0

  trainCorrect = 0
  valCorrect = 0

  for (x, y) in train_loader:
    (x,y) = (x.to(device), y.to(device))

    pred = custom_model(x)
    loss = loss_func(pred, y)

    opt.zero_grad()
    loss.backward()
    opt.step()

    totalTrainLoss += loss
    trainCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

  print('Train loss for the epoch'+ str(e)+'is:',totalTrainLoss)
  print('Number of correctly predicted outputs for the epoch' + str(e)+ 'is:', trainCorrect)

Train loss for the epoch0is: tensor(10.5663, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch0is: 1132.0
Train loss for the epoch1is: tensor(6.1281, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch1is: 1201.0
Train loss for the epoch2is: tensor(5.2122, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch2is: 1205.0
Train loss for the epoch3is: tensor(3.2544, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch3is: 1238.0
Train loss for the epoch4is: tensor(4.3221, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch4is: 1231.0
Train loss for the epoch5is: tensor(3.2306, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predicted outputs for the epoch5is: 1240.0
Train loss for the epoch6is: tensor(2.7459, device='cuda:0', grad_fn=<AddBackward0>)
Number of correctly predic